In [22]:
import pandas as pd
import warnings
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

In [23]:
gp_round = 8
dict_nb_laps = {1: 57, 2: 50, 3: 58, 4: 63, 5: 57, 6: 66, 7: 78, 8: 51, 9: 70, 10: 52, 11: 71, 12: 53, 13: 70, 14: 44, 15: 72, 16: 53, 17: 61, 18: 53, 19: 56, 20: 71, 21: 71, 22: 58}
dict_pit_stops_time = {1: 25, 2: 21, 3: 22, 4: 25, 5: 25, 6: 22, 7: 24, 8: 20, 9: 24, 10: 29, 11: 21, 12: 30, 13: 22, 14: 23, 15: 19, 16: 25, 17: 29, 18: 24, 19: 24, 20: 23, 21: 23, 22: 22}

In [24]:
# Course ??
tourTot = dict_nb_laps[gp_round]
perteArret = dict_pit_stops_time[gp_round]

In [25]:
df_combi = pd.DataFrame(columns=['Train1', 'Train2', 'Train3'])

for i in range(1,tourTot-1):
    T1 = i
    for j in range(1, tourTot-1):        
        if (i+j) > tourTot-1:
            continue
        T2 = j
        T3 = tourTot - j - i
        df_combi = df_combi.append({'Train1' : T1, 'Train2' : T2, 'Train3' : T3}, ignore_index = True)
        
df_combi

,Train1,Train2,Train3
0,1,1,49
1,1,2,48
2,1,3,47
3,1,4,46
4,1,5,45
...,...,...,...
1220,47,2,2
1221,47,3,1
1222,48,1,2
1223,48,2,1


In [26]:
df_combi2 = pd.DataFrame(columns=['Train1', 'Train2'])

for i in range(1,tourTot):
    T1 = i
    T2 = tourTot-i
    df_combi2 = df_combi2.append({'Train1' : T1, 'Train2' : T2}, ignore_index = True)
    
df_combi2.head()

,Train1,Train2
0,1,50
1,2,49
2,3,48
3,4,47
4,5,46


In [27]:
dffp_complet = pd.read_csv(f'../data/prediction_times_round_{gp_round}.csv', index_col=0)
dffp_complet.head()

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime,TimeStr
0,1,SOFT,107.869088,0.435713,0.084775,108.304801,108.220026,01:48.220
1,2,SOFT,107.869088,0.487998,0.169550,108.357087,108.187537,01:48.187
2,3,SOFT,107.869088,0.546558,0.254325,108.415647,108.161321,01:48.161
3,4,SOFT,107.869088,0.612145,0.339100,108.481234,108.142133,01:48.142
4,5,SOFT,107.869088,0.685603,0.423875,108.554691,108.130816,01:48.130


In [28]:
dffp_soft = dffp_complet[dffp_complet['Tyre']=='SOFT'].copy()
dffp_soft.drop(columns = ['Tyre', 'LapTimeSeconds', 'DeltaDeg', 'TimeStr', 'FinalLapTime'], inplace = True)
dffp_soft.rename(columns={'AdjustedTime': 'S', 'Lap': 'TyreLife'}, inplace = True)

dffp_medium = dffp_complet[dffp_complet['Tyre']=='MEDIUM'].copy()
dffp_medium.drop(columns = ['Tyre', 'LapTimeSeconds', 'DeltaDeg', 'TimeStr', 'FinalLapTime'], inplace = True)
dffp_medium.rename(columns={'AdjustedTime': 'M', 'Lap': 'TyreLife'}, inplace = True)

dffp_hard = dffp_complet[dffp_complet['Tyre']=='HARD'].copy()
dffp_hard.drop(columns = ['Tyre', 'LapTimeSeconds', 'DeltaDeg', 'TimeStr' ,'FinalLapTime'], inplace = True)
dffp_hard.rename(columns={'AdjustedTime': 'H', 'Lap': 'TyreLife'}, inplace = True)

In [29]:
dffp_final = dffp_soft.merge(dffp_medium, how = 'inner', left_on = 'TyreLife', right_on = 'TyreLife')
dffp_final = dffp_final.merge(dffp_hard, how = 'inner', left_on = 'TyreLife', right_on = 'TyreLife')
dffp_final.drop(columns = ['FuelDeg_y', 'FuelDeg_x'], inplace = True)
dffp_final = dffp_final[['TyreLife', 'S', 'M', 'H', 'FuelDeg']]

dffp_final

,TyreLife,S,M,H,FuelDeg
0,1,108.304801,109.142409,109.457109,0.084775
1,2,108.357087,109.151947,109.464391,0.169550
2,3,108.415647,109.162342,109.472183,0.254325
3,4,108.481234,109.173674,109.480520,0.339100
4,5,108.554691,109.186025,109.489440,0.423875
5,6,108.636963,109.199488,109.498985,0.508651
6,7,108.729108,109.214162,109.509199,0.593426
7,8,108.832311,109.230157,109.520127,0.678201
8,9,108.947897,109.247592,109.531820,0.762976
9,10,109.077355,109.266596,109.544331,0.847751


In [30]:
#Ouvre le csv avec les combinaisons pour 1 arrêts
tyreCombi2 = pd.read_csv('temp/TyreCombi2.csv', sep = ";")


#Ouvre le csv avec les combinaisons pour 2 arrêts
tyreCombi3 = pd.read_csv('temp/TyreCombi3.csv', sep = ";")

In [31]:
tyreCombi2

,P1,P2
0,S,M
1,S,H
2,M,S
3,M,H


In [32]:
tyreCombi3

,P1,P2,P3
0,S,S,M
1,S,M,M
2,M,M,S
3,M,S,S
4,S,M,S


In [33]:
# Boucle sur tous les types de pneu pour 2 arrêts afin de générer tous les temps totaux pour la course

time1 = 0
time2 = 0
time3 = 0
listeT = []
# liste_combi = []
df_timerace3 = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeraceter = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timerace3 = df_timeraceter

In [34]:
#Boucle sur le df tyre_combi

for a in range(len(tyreCombi3)):
    lig = a
    df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])

    # Calcul du T1 en fonction du type de pneu 
    col = 0
    liste_combi = []
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
         
    #Boucle sur la longueur du df
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,0]
            
        #Addition des nbTours premières lignes
            
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time1 += dffp_final.iloc[i,colT]      
            
        df_timeracebis = df_timeracebis.append({'TimeT1' : time1}, ignore_index = True)
        df_timeracebis.iloc[l,1] = nbTour
        time1 = 0
        
        
# Calcul du T2 en fonction du type de pneu        
    col = 1
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
        
    #Boucle sur la longueur du df    
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,1]
        
        #Addition des nbTours premières lignes    
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time2 = time2 + dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,5] = time2
        df_timeracebis.iloc[l,2] = nbTour
        time2 = 0
        
        
# Calcul du T3 en fonction du type de pneu    
    
    col = 2
    liste_combi.append(tyreCombi3.iloc[lig,col])
    
    if tyreCombi3.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi3.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
    
    #Boucle sur la longueur du df    
    for l in range(len(df_combi)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi.iloc[l,2]
        
        #Addition des nbTours premières lignes
        
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time3 = time3 + dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,6]=time3
        df_timeracebis.iloc[l,3] = nbTour
        time3 = 0
        
        df_timeracebis.iloc[l,0] = "".join(liste_combi)

    df_timerace3 = pd.concat([df_timerace3,df_timeracebis], ignore_index = True)

In [35]:
#Calcul du temps total
df_timerace3['Total_time'] = df_timerace3['TimeT1'] + df_timerace3['TimeT2'] + df_timerace3['TimeT3'] + ( 2 * perteArret)
df_timerace3

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,SSM,1,1,49,108.304801,108.304801,5421.931616,5678.541219
1,SSM,1,2,48,108.304801,216.661888,5306.262954,5671.229643
2,SSM,1,3,47,108.304801,325.077534,5191.141907,5664.524243
3,SSM,1,4,46,108.304801,433.558768,5076.523259,5658.386828
4,SSM,1,5,45,108.304801,542.113459,4962.365527,5652.783787
...,...,...,...,...,...,...,...,...
6120,SMS,47,2,2,5813.122774,218.294356,216.661888,6288.079018
6121,SMS,47,3,1,5813.122774,327.456698,108.304801,6288.884274
6122,SMS,48,1,2,6010.620651,109.142409,216.661888,6376.424948
6123,SMS,48,2,1,6010.620651,218.294356,108.304801,6377.219808


In [36]:
time1 = 0
time2 = 0
time3 = 0
listeT = []
# liste_combi = []
df_timerace2 = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracebis = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timeracequa = pd.DataFrame(columns=['combiTyre','Train1', 'Train2', 'Train3','TimeT1', 'TimeT2', 'TimeT3', 'Total_time'])
df_timerace2 = df_timeracequa

In [37]:
#Boucle sur le df tyre_combi

for a in range(0,len(tyreCombi2)):
    lig = a
    df_timeracebis = df_timeraceter

    # Calcul du T1 en fonction du type de pneu 
    col = 0
    liste_combi = []
    liste_combi.append(tyreCombi2.iloc[lig,col])
    
    if tyreCombi2.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi2.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
         
    #Boucle sur la longueur du df
    for l in range(len(df_combi2)):
        #nombre de tour égal à la valeur de la 1ère colonne
        nbTour = df_combi2.iloc[l,0]
            
        #Addition des nbTours premières lignes
            
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time1 = time1 + dffp_final.iloc[i,colT]      
            
        df_timeracebis = df_timeracebis.append({'TimeT1' : time1}, ignore_index = True)
        df_timeracebis.iloc[l,1] = nbTour
        time1 = 0
        
        
# Calcul du T2 en fonction du type de pneu          
        
    col = 1
    liste_combi.append(tyreCombi2.iloc[lig,col])
    
    if tyreCombi2.iloc[lig,col] == 'S':
        colT = 1
    elif tyreCombi2.iloc[lig,col] == 'M':
        colT = 2
    else :
        colT = 3
        
        
    #Boucle sur la longueur du df    
    for l in range(len(df_combi2)):
        #nombre de tour égal à la valeur de la 2ème colonne
        nbTour = df_combi2.iloc[l,1]
        
        #Addition des nbTours premières lignes
        
        for i in range(nbTour): #J'itère autant de fois qu'il y a de tours
            time2 += dffp_final.iloc[i,colT]
            
        df_timeracebis.iloc[l,5] = time2
        df_timeracebis.iloc[l,2] = nbTour
        time2 = 0
        
        
# Génération du Train3 et Time3 à 0    
  
    
    #Boucle sur la longueur du df    
    for l in range(len(df_combi2)):        
            
        df_timeracebis.iloc[l,6]=0
        df_timeracebis.iloc[l,3] = 0
        
        
        df_timeracebis.iloc[l,0] = "".join(liste_combi)

    df_timerace2 = pd.concat([df_timerace2,df_timeracebis], ignore_index = True)

In [38]:
#Calcul du temsp total
df_timerace2['Total_time'] = df_timerace2['TimeT1'] + df_timerace2['TimeT2'] + df_timerace2['TimeT3']  + ( perteArret)
df_timerace2

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,SM,1,50,0,108.304801,5538.197178,0,5666.501979
1,SM,2,49,0,216.661888,5421.931616,0,5658.593504
2,SM,3,48,0,325.077534,5306.262954,0,5651.340489
3,SM,4,47,0,433.558768,5191.141907,0,5644.700675
4,SM,5,46,0,542.113459,5076.523259,0,5638.636718
...,...,...,...,...,...,...,...,...
195,MH,46,5,0,5076.523259,547.363643,0,5643.886902
196,MH,47,4,0,5191.141907,437.874203,0,5649.01611
197,MH,48,3,0,5306.262954,328.393683,0,5654.656637
198,MH,49,2,0,5421.931616,218.9215,0,5660.853117


In [39]:
#Concatène les 2 df
df_timerace2 = df_timerace2.sort_values(by='Total_time', ascending=True)

df_timerace3 = df_timerace3.sort_values(by='Total_time', ascending=True)


df_timerace = pd.concat([df_timerace2,df_timerace3], ignore_index = True)

# Affiche les 10 combinaisons les plus performantes
# df_timerace = df_timerace.sort_values('Total_time', ascending=True)

df_timerace

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,MS,34,17,0,3728.113327,1855.073749,0,5603.187077
1,SM,17,34,0,1855.073749,3728.113327,0,5603.187077
2,SM,18,33,0,1965.934461,3617.256091,0,5603.190552
3,MS,33,18,0,3617.256091,1965.934461,0,5603.190552
4,SM,16,35,0,1744.533569,3839.134435,0,5603.668004
...,...,...,...,...,...,...,...,...
6320,MSS,1,49,1,109.142409,6218.873982,108.304801,6476.321192
6321,SMS,1,1,49,108.304801,109.142409,6218.873982,6476.321192
6322,MSS,1,1,49,109.142409,108.304801,6218.873982,6476.321192
6323,SMM,49,1,1,6218.873982,109.142409,109.142409,6477.1588


In [40]:
total_fuel = dffp_final['FuelDeg'].sum()
df_timerace['Total_time'] = df_timerace['Total_time'] - total_fuel
df_timerace

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,MS,34,17,0,3728.113327,1855.073749,0,5490.775312
1,SM,17,34,0,1855.073749,3728.113327,0,5490.775312
2,SM,18,33,0,1965.934461,3617.256091,0,5490.778787
3,MS,33,18,0,3617.256091,1965.934461,0,5490.778787
4,SM,16,35,0,1744.533569,3839.134435,0,5491.25624
...,...,...,...,...,...,...,...,...
6320,MSS,1,49,1,109.142409,6218.873982,108.304801,6363.909427
6321,SMS,1,1,49,108.304801,109.142409,6218.873982,6363.909427
6322,MSS,1,1,49,109.142409,108.304801,6218.873982,6363.909427
6323,SMM,49,1,1,6218.873982,109.142409,109.142409,6364.747035


In [41]:
df_final = pd.concat([df_timerace[df_timerace['Train3'] == 0].iloc[:2], df_timerace[df_timerace['Train3'] != 0].iloc[:2]]).reset_index(drop=True).drop([1, 3], axis=0).reset_index(drop=True).sort_values(by='Total_time', ascending=True)
df_final

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time
0,MS,34,17,0,3728.113327,1855.073749,0,5490.775312
1,SMS,14,23,14,1524.281111,2515.206452,1524.281111,5491.35691


In [42]:
df_final['TotalTimeStr'] = df_final['Total_time'].apply(lambda x: str(datetime.timedelta(seconds=x))[:-3])
df_final

,combiTyre,Train1,Train2,Train3,TimeT1,TimeT2,TimeT3,Total_time,TotalTimeStr
0,MS,34,17,0,3728.113327,1855.073749,0,5490.775312,1:31:30.775
1,SMS,14,23,14,1524.281111,2515.206452,1524.281111,5491.35691,1:31:31.356


In [43]:
df_final.to_csv(f'../data/predicted_strategy_round_{gp_round}.csv')